## Google colab setup

In [ ]:
try:
  import google.colab
  IS_COLAB = True
except:
  IS_COLAB = False

In [ ]:
if IS_COLAB:
  !pip install tensorflow_addons
  !pip install unidecode
  !pip install transformers
  !pip install nlpaug

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/SemiSupervised/*.csv .

## Imports and setup

In [1]:
%%capture
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import re
import time
import spacy
import numpy as np
from transformers import AutoTokenizer, TFAutoModel
from unidecode import unidecode
from utils import prepare_ds, prepare_train_ds

2023-04-29 08:46:17.251496: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 08:46:17.775502: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda/extras/CUPTI/lib64::/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda/extras/CUPTI/lib64:
2023-04-29 08:46:17.775549: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object 

In [2]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

## Prepare data splits

In [3]:
labeled_ds, unlabeled_ds = prepare_train_ds("train_ner.csv")
test_ds = prepare_ds("test_ner.csv")
val_ds = prepare_ds("validation_internal_ner.csv")

2023-04-29 08:47:04.372767: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 08:47:04.373564: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-29 08:47:04.373921: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-29 08:47:04.374221: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

## Model definition and declaration

In [4]:
class FixMatchTune(tf.keras.Model):
  def __init__(
      self,
      encoder_name="readerbench/RoBERT-base",
      num_classes=4,
      **kwargs
  ):
    super(FixMatchTune,self).__init__(**kwargs)

    self.bert = TFAutoModel.from_pretrained(encoder_name)
    self.num_classes = num_classes
    self.weak_augment = tf.keras.layers.GaussianNoise(stddev=0.5)
    self.strong_augment = tf.keras.layers.GaussianNoise(stddev=5)

    self.cls_head = tf.keras.Sequential([
      tf.keras.layers.Dense(256,activation="relu"),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(64,activation="relu"),
      tf.keras.layers.Dense(self.num_classes, activation="softmax")
    ])

  def call(self, inputs, training):
    ids, mask = inputs
    
    embeds = self.bert(input_ids=ids, attention_mask=mask,training=training).pooler_output

    strongs = self.strong_augment(embeds,training=training)
    weaks = self.weak_augment(embeds,training=training)

    strong_preds = self.cls_head(strongs,training=training)
    weak_preds = self.cls_head(weaks,training=training)

    return weak_preds, strong_preds
  
class FixMatch(tf.keras.Model):
  def __init__(
      self,
      encoder_name="readerbench/RoBERT-base",
      num_classes=4,
      encoder_train=False,
      **kwargs
  ):
    super(FixMatch,self).__init__(**kwargs)

    self.bert = TFAutoModel.from_pretrained(encoder_name)
    self.bert.trainable = encoder_train
    self.num_classes = num_classes
    self.weak_augment = tf.keras.layers.GaussianNoise(stddev=0.5)
    self.strong_augment = tf.keras.layers.GaussianNoise(stddev=5)


    self.cls_head = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(10),
            merge_mode="ave"
        ),
        tf.keras.layers.Dense(units=self.num_classes,activation="softmax")
    ])

  def call(self, inputs, training):
    ids, mask = inputs
    
    embeds = self.bert(input_ids=ids, attention_mask=mask).last_hidden_state

    strongs = self.strong_augment(embeds,training=training)
    weaks = self.weak_augment(embeds,training=training)

    strong_preds = self.cls_head(strongs)
    weak_preds = self.cls_head(weaks)

    return weak_preds, strong_preds



In [5]:
# model = FixMatch()
model = FixMatchTune()

## Train and evaluate

In [7]:
optim = tfa.optimizers.AdamW(weight_decay=0.001,learning_rate=0.005)
optim2 = tfa.optimizers.AdamW(weight_decay=0.0,learning_rate=0.00001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_prec_metric = tf.keras.metrics.Precision(name="precision")
val_recall_metric = tf.keras.metrics.Recall(name="recall")
f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')

In [8]:
thresh = 0.9
unsup_weight = 1.0
steps_per_epoch = 600

In [9]:
@tf.function
def train_step_tune(x,y,xu):

  with tf.GradientTape() as tape:
      wl,_ = model([x["input_ids"],x["attention_mask"]],training=True)
      wu,su = model([xu["input_ids"],xu["attention_mask"]],training=True)
      ls = loss_fn(y, wl)
      mask = tf.reduce_max(wu,axis=1)>thresh
      wu = wu[mask]
      su = su[mask]
      lu = loss_fn(tf.argmax(wu,axis=1),su)

      loss = ls + unsup_weight * lu

  grads = tape.gradient(loss, [model.cls_head.trainable_weights, model.bert.trainable_weights])
  optim.apply_gradients(zip(grads[0], model.cls_head.trainable_weights))
  optim2.apply_gradients(zip(grads[1], model.bert.trainable_weights))

  return loss

@tf.function
def train_step(x,y,xu):

  with tf.GradientTape() as tape:
      wl,_ = model([x["input_ids"],x["attention_mask"]],training=True)
      wu,su = model([xu["input_ids"],xu["attention_mask"]],training=True)
      ls = loss_fn(y, wl)
      mask = tf.reduce_max(wu,axis=1)>thresh
      wu = wu[mask]
      su = su[mask]
      lu = loss_fn(tf.argmax(wu,axis=1),su)

      loss = ls + unsup_weight * lu

  grads = tape.gradient(loss, model.trainable_weights)
  optim.apply_gradients(zip(grads, model.trainable_weights))

  return loss

@tf.function
def test_step(x, y):
    wpred,_ = model([x["input_ids"],x["attention_mask"]], training=False)
    val_acc_metric.update_state(y, wpred)
    true_hot = tf.one_hot(y, 4)
    val_prec_metric.update_state(true_hot, wpred)
    val_recall_metric.update_state(true_hot, wpred)
    f1_metric_micro.update_state(true_hot, wpred)
    f1_metric_macro.update_state(true_hot, wpred)

In [10]:
EPOCHS=16
max_val_acc = 0.0
best_weights = None

l_iter = iter(labeled_ds)
u_iter = iter(unlabeled_ds)
for epoch in range(EPOCHS):
  print(f"Epoch {epoch}")
  start_time = time.time()
  for step in range(steps_per_epoch):
    x, y = next(l_iter)
    xu = next(u_iter)
    
    loss = train_step_tune(x,y,xu)
    if step % 50 == 0:
      print(
        "Training loss (for one batch) at step %d: %.4f"
        % (step, float(loss))
      )
  for x_batch_val, y_batch_val in val_ds:
      test_step(x_batch_val, y_batch_val)

  acc = float(val_acc_metric.result())
  prec = float(val_prec_metric.result())
  recall = float(val_recall_metric.result())
  micro = float(f1_metric_micro.result())
  macro = float(f1_metric_macro.result())

  val_acc_metric.reset_states()
  val_prec_metric.reset_states()
  val_recall_metric.reset_states()
  f1_metric_micro.reset_states()
  f1_metric_macro.reset_states()

  if acc > max_val_acc:
    max_val_acc = acc
    best_weights = model.get_weights()
  print(f"Validation acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")
  print("Time taken: %.2fs" % (time.time() - start_time))
print("Restoring best weights relative to validation accuracy...")
model.set_weights(best_weights)

Epoch 0
Training loss (for one batch) at step 0: 1.7838
Training loss (for one batch) at step 50: 1.2648
Training loss (for one batch) at step 100: 1.1871
Training loss (for one batch) at step 150: 1.0920
Training loss (for one batch) at step 200: 0.5658
Training loss (for one batch) at step 250: 5.4454
Training loss (for one batch) at step 300: 0.7656
Training loss (for one batch) at step 350: 0.7228
Training loss (for one batch) at step 400: 6.4009
Training loss (for one batch) at step 450: 0.7365
Training loss (for one batch) at step 500: 1.3853
Training loss (for one batch) at step 550: 0.6015
Validation acc: 0.583 precision: 0.604 recall: 0.572 f1_micro: 0.588 f1_macro: 0.524
Time taken: 228.88s
Epoch 1
Training loss (for one batch) at step 0: 0.2515
Training loss (for one batch) at step 50: 0.5211
Training loss (for one batch) at step 100: 0.2893
Training loss (for one batch) at step 150: 0.4689
Training loss (for one batch) at step 200: 0.5770
Training loss (for one batch) at st

In [11]:
for x_batch_val, y_batch_val in test_ds:
    test_step(x_batch_val, y_batch_val)
acc = float(val_acc_metric.result())
prec = float(val_prec_metric.result())
recall = float(val_recall_metric.result())
micro = float(f1_metric_micro.result())
macro = float(f1_metric_macro.result())

val_acc_metric.reset_states()
val_prec_metric.reset_states()
val_recall_metric.reset_states()
f1_metric_micro.reset_states()
f1_metric_macro.reset_states()
print(f"Test acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")

Test acc: 0.687 precision: 0.687 recall: 0.687 f1_micro: 0.687 f1_macro: 0.680


In [12]:
model.save_weights("./checkpoints/fixmatch_tune")